In [6]:
import glob, h5py, os
import numpy as np
import matplotlib.pyplot as plt

from oc_ica import gabor_fit, plotting, analysis

In [2]:
folder = '/home/jesse/Development/results/oc_ica/nat_images_mixtures-64_sources-128/'
files = sorted(glob.glob(os.path.join(folder, '*.h5')))
models = [f.split('.')[0].split('-')[-1] for f in files]
n_mixtures = int(folder.split('-')[-2].split('_')[0])
n_sources = int(folder.split('-')[-1].split('_')[0][:-1])
print(models)

['2', '3', '4', 'COHERENCE_SOFT', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F']


In [4]:
ws = np.full((len(models), n_sources, n_mixtures), np.nan)
for ii, fname in enumerate(files):
    with h5py.File(fname) as f:
        ws[ii] = f['W_fits'].value

In [5]:
params = plotting.get_Gabor_params(ws)

In [9]:
reload(plotting)
#plotting.plot_GaborFit_xy(params[0][1], int(np.sqrt(n_mixtures)))
plotting.plot_GaborFit_polar(params[0][1])
#plotting.plot_GaborFit_envelope(params[0][1])
plt.show()

In [51]:
fitter = gabor_fit.GaborFit()

In [74]:
reload(plotting)
plotting.plot_bases(ws[0])
wp = fitter.make_gabor(params[0][0][-1], 8, 8).T
wp = wp.reshape(wp.shape[0], -1)
plotting.plot_bases(wp)
plt.show()

In [25]:
reload(plotting)
idxs = [0, 4, 6, 2]
w = ws[idxs]
plotting.plot_figure3(w)

In [9]:
ws.shape

(8, 128, 64)